In [1]:
import os
import gc
import random
import warnings
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.amp import GradScaler, autocast
from torch.optim.swa_utils import AveragedModel, SWALR, update_bn
from sklearn.model_selection import StratifiedKFold
from datetime import datetime
import mlflow
from tqdm import tqdm
import timm
from sklearn.metrics import log_loss
import albumentations as A

# Import your existing modules
from config import load_config
from models.model import BaseModel
from datasets.custom_dataset import CustomImageDataset
from transforms.aug import get_transforms, get_tta_transforms, TTAWrapper
from train_utils.trainer import train_one_epoch, validate_one_epoch
from train_utils.mlflow_cb import MLflowCallback
from train_utils.option import get_optimizer, get_scheduler, get_criterion
from utils.common import seed_everything, get_device

In [2]:
warnings.filterwarnings('ignore')

In [3]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

class CombinedLoss(nn.Module):
    def __init__(self, loss_fn1, loss_fn2, weight1=1.0, weight2=1.0, reduction='mean'):
        super(CombinedLoss, self).__init__()
        self.loss_fn1 = loss_fn1
        self.loss_fn2 = loss_fn2
        self.weight1 = weight1
        self.weight2 = weight2
        self.reduction = reduction

    def forward(self, inputs, targets):
        loss1 = self.loss_fn1(inputs, targets)
        loss2 = self.loss_fn2(inputs, targets)
        combined = self.weight1 * loss1 + self.weight2 * loss2

        if self.reduction == 'mean':
            return combined.mean() if combined.dim() > 0 else combined
        elif self.reduction == 'sum':
            return combined.sum()
        else:
            return combined


In [4]:
def train_fold(fold_idx, train_data, val_data, hp, exp, device, num_classes, run_name):
    print(f"\n{'='*50}")
    print(f"Training Fold {fold_idx + 1}")
    print(f"\n{'='*50}")

    best_val_loss = float('inf')

    model = BaseModel(
        exp['model_name'],
        num_classes,
        exp['embedding_dim'],
    )

    model = model.to(device)

    optimizer = get_optimizer(model, hp)
    scheduler = get_scheduler(optimizer, hp)
    swa_start   = hp.get('swa_start', hp['epochs'] - 5)
    swa_lr      = hp.get('swa_lr', 1e-4)
    swa_model   = AveragedModel(model) 
    swa_scheduler = SWALR(optimizer, swa_lr=swa_lr)
    
    # criterion = FocalLoss()
    # criterion = get_criterion(hp)

    focal_loss_fn = FocalLoss()
    cross_entropy_fn = nn.CrossEntropyLoss()
    criterion = CombinedLoss(
        loss_fn1=focal_loss_fn,
        loss_fn2=cross_entropy_fn,
        # weight1=0.5,  # focal loss 비중
        # weight2=0.5,  # cross entropy 비중
        # reduction='mean'
    )
    
    scaler = GradScaler()

    train_loader = DataLoader(
        CustomImageDataset(train_data, transform=get_transforms(hp['img_size'])),
        batch_size=hp['batch_size'], shuffle=True, num_workers=hp['cpu_count'], pin_memory=True
    )
    val_loader = DataLoader(
        CustomImageDataset(val_data, transform=get_transforms(hp['img_size'])),
        batch_size=hp['batch_size'], shuffle=False, num_workers=hp['cpu_count'], pin_memory=True
    )

    for epoch in range(1, hp['epochs'] + 1):
        train_logloss, train_acc, _, _ = train_one_epoch(
            model, train_loader, criterion,
            optimizer, scaler, device,
            use_mixup=False, use_cutmix=hp.get('use_cutmix', False)
        )

        scheduler.step()
        if epoch < swa_start:
            scheduler.step()
        else:
            swa_model.update_parameters(model)
            swa_scheduler.step()

        val_logloss, val_acc, val_probs_np, val_labels_np = validate_one_epoch(
            model, val_loader, criterion, device
        )

        mlflow.log_metric("train_logloss", train_logloss, step=epoch)
        mlflow.log_metric("train_accuracy", train_acc, step=epoch)
        mlflow.log_metric("val_logloss", val_logloss, step=epoch)
        mlflow.log_metric("val_accuracy", val_acc, step=epoch)

        print(f"Epoch {epoch} ▶ Train LL: {train_logloss:.4f} / Val LL: {val_logloss:.4f} | Train Acc: {train_acc:.4f} / Val Acc: {val_acc:.4f}")

        if val_logloss < best_val_loss:
            best_val_loss = val_logloss
            torch.save(model.state_dict(), f"best_model_fold_{fold_idx}.pth")

    best_model = BaseModel(exp['model_name'], num_classes, exp['embedding_dim']).to(device)
    best_model.load_state_dict(torch.load(f"best_model_fold_{fold_idx}.pth"))
    update_bn(train_loader, swa_model, device=device)
    torch.save(swa_model.state_dict(), f"swa_model_fold_{fold_idx}.pth")
    best_model = swa_model
    
    return best_model, best_val_loss

In [5]:
# 설정 로드
config_path = "config.yaml"
hp, exp, mf = load_config(config_path)

# 환경 설정
seed_everything(exp['seed'])
device = get_device()

# MLflow 설정
mlflow.set_tracking_uri(mf['mlflow_tracking_uri'])
mlflow.set_experiment(exp['name'])
run_name = datetime.now().strftime("%Y%m%d_%H%M%S")
mlflow.start_run(run_name=run_name)
print(f"MLflow experiment [{exp['name']}] run [{run_name}] started.")

MLflow experiment [hecto_image_classification] run [20250614_190004] started.


In [6]:
mlflow.log_params(hp)
mlflow.log_params(exp)
mlflow.log_params({
    'torch_version': torch.__version__,
    'timm_version': timm.__version__,
    'cuda_available': torch.cuda.is_available(),
    'cuda_device': torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu"
})

In [7]:
print("Loading train data...")
# 데이터 로딩 및 전처리
train_df = pd.read_csv('./dataset/train.csv', index_col=0, encoding='utf-8-sig').drop_duplicates(subset=['hash'], keep='first')

# 클래스명 매핑
class_alias_map = {
    'K5_하이브리드_3세대_2020_2023': 'K5_3세대_하이브리드_2020_2022',
    '디_올_뉴_니로_2022_2025': '디_올뉴니로_2022_2025',
    '박스터_718_2017_2024': '718_박스터_2017_2024',
    '라브4_4세대_2013_2018': 'RAV4_2016_2018',
    '라브4_5세대_2019_2024': 'RAV4_5세대_2019_2024',
}

train_df['label'] = train_df['label'].apply(lambda x: class_alias_map.get(x, x))

# 클래스 매핑
classes = sorted(train_df['label'].unique())
class_to_idx = {c: i for i, c in enumerate(classes)}
train_df['label'] = train_df['label'].map(class_to_idx)
num_classes = len(classes)

mlflow.log_param("n_classes", num_classes)
print(f"Total classes: {num_classes}, Total samples: {len(train_df)}")

Loading train data...
Total classes: 391, Total samples: 31426


In [8]:
n_folds = exp.get('n_folds', 5)
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=exp['seed'])

fold_models = []
fold_scores = []

for fold_idx, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['label'])):
    with mlflow.start_run(run_name=f"fold{fold_idx+1}", nested=True):
        print(f"\n[FOLD {fold_idx+1}] training started.")
        mlflow.log_param("fold_idx", fold_idx)
        # (파라미터 로깅, 데이터 나누기 등)
        train_data = train_df.iloc[train_idx].reset_index(drop=True)
        val_data = train_df.iloc[val_idx].reset_index(drop=True)

        model, val_loss = train_fold(
            fold_idx, train_data, val_data,
            hp, exp, device,
            num_classes, run_name
        )
        mlflow.log_metric("val_logloss", val_loss)
        print(f"[FOLD {fold_idx+1}] validation logloss: {val_loss:.5f}")
        torch.save(model.state_dict(), f"best_model_fold_{fold_idx}.pth")
        mlflow.log_artifact(f"best_model_fold_{fold_idx}.pth")

        fold_models.append(model)
        fold_scores.append(1.0 / (1.0 + val_loss))

    gc.collect()
    torch.cuda.empty_cache()

# [Fold별 가중치 계산 및 로깅]
total_score = sum(fold_scores)
fold_weights = [score / total_score for score in fold_scores]
mlflow.log_params({'n_folds': n_folds, 'fold_weights': fold_weights})
print(f"Fold weights: {fold_weights}")


[FOLD 1] training started.

Training Fold 1



Valid: 100%|██████████| 197/197 [00:11<00:00, 17.32it/s]


Epoch 1 ▶ Train LL: 3.7932 / Val LL: 0.6626 | Train Acc: 0.3523 / Val Acc: 0.8910


Valid: 100%|██████████| 197/197 [00:11<00:00, 17.27it/s]


Epoch 2 ▶ Train LL: 1.7975 / Val LL: 0.2703 | Train Acc: 0.6646 / Val Acc: 0.9370


...

Epoch 28 ▶ Train LL: 0.8200 / Val LL: 0.0822 | Train Acc: 0.7207 / Val Acc: 0.9742


Valid: 100%|██████████| 197/197 [00:12<00:00, 16.40it/s]


Epoch 29 ▶ Train LL: 0.8195 / Val LL: 0.0777 | Train Acc: 0.7220 / Val Acc: 0.9752


Valid: 100%|██████████| 197/197 [00:11<00:00, 16.48it/s]


Epoch 30 ▶ Train LL: 0.7787 / Val LL: 0.0745 | Train Acc: 0.7325 / Val Acc: 0.9758
[FOLD 5] validation logloss: 0.06034
🏃 View run fold5 at: https://6m9uma7tvgsttm-5000.proxy.runpod.net/#/experiments/2/runs/e1b71f25b8ec424393995589e32f4f6e
🧪 View experiment at: https://6m9uma7tvgsttm-5000.proxy.runpod.net/#/experiments/2
Fold weights: [0.19991306922666785, 0.1998462790902399, 0.19895187176208082, 0.2006598992845391, 0.20062888063647247]


In [ ]:
import torch

# [테스트셋 추론 및 결과 저장]
print("Test inference started...")


resize_only = A.Compose([A.Resize(hp['img_size'], hp['img_size'])])

# 테스트 데이터 로드 및 전처리
test_df = pd.read_csv('./dataset/test.csv')
test_df['img_path'] = './dataset/' + test_df['img_path']

test_dataset = CustomImageDataset(test_df, transform=resize_only, is_test=True)
test_loader = DataLoader(
    test_dataset,
    batch_size=hp['batch_size'], shuffle=False, num_workers=hp['cpu_count'], pin_memory=True
)

# TTA 변환 정의
tta_transforms = get_tta_transforms()

print(f"\nPerforming inference with TTA ({len(tta_transforms)+1} augmentations)...")

# 모든 fold 모델을 사용하여 TTA 예측
all_fold_predictions = []

for fold_idx, model in enumerate(fold_models):
    print(f"Processing fold {fold_idx + 1}/{n_folds}...")
    model.eval()
    tta_model = TTAWrapper(model, tta_transforms, device)
    fold_predictions = []
    
    with torch.no_grad():
        for images in tqdm(test_loader, desc=f"Fold {fold_idx + 1} TTA"):
            imgs_np = images.numpy()
            mean_prob = tta_model.predict(imgs_np)
            fold_predictions.append(mean_prob)
            
    # Fold 예측 결과 저장
    fold_pred_np = torch.cat(fold_predictions).cpu().numpy()
    all_fold_predictions.append(fold_pred_np)

# Fold별 가중치를 적용한 최종 앙상블 예측
print("\nCombining fold predictions with weights...")

weighted_predictions = np.zeros_like(all_fold_predictions[0])
for fold_pred, weight in zip(all_fold_predictions, fold_weights):
    weighted_predictions += fold_pred * weight

# 예측 결과를 DataFrame으로 변환
sorted_class_names = sorted(class_to_idx.keys(), key=lambda k: class_to_idx[k])
pred_df = pd.DataFrame(weighted_predictions, columns=sorted_class_names)

Test inference started...

Performing inference with TTA (4 augmentations)...
Processing fold 1/5...


Fold 1 TTA: 100%|██████████| 259/259 [03:14<00:00,  1.33it/s]


Processing fold 2/5...


Fold 2 TTA: 100%|██████████| 259/259 [03:14<00:00,  1.33it/s]


Processing fold 3/5...


Fold 3 TTA: 100%|██████████| 259/259 [03:14<00:00,  1.33it/s]


Processing fold 4/5...


Fold 4 TTA: 100%|██████████| 259/259 [03:14<00:00,  1.33it/s]


Processing fold 5/5...


Fold 5 TTA:  29%|██▊       | 74/259 [00:56<02:17,  1.34it/s]

In [ ]:
# Submission 파일 생성
submission = pd.read_csv('./dataset/sample_submission.csv', encoding='utf-8-sig')
submission_class_columns = submission.columns[1:]

# 누락된 클래스 컬럼은 0으로 추가
for col in submission_class_columns:
    if col not in pred_df.columns:
        pred_df[col] = 0.0

# 결과 업데이트
submission[pred_df.columns] = pred_df.values

# 최종 결과 저장
submission.to_csv('submission.csv', index=False, encoding='utf-8-sig')

submission.to_csv('submission.csv', index=False, encoding='utf-8-sig')
mlflow.log_artifact('submission.csv')
print("Submission file 'submission.csv' created and logged to MLflow.")

mlflow.log_artifact("config.yaml")
mlflow.log_artifact("models/")
mlflow.log_artifact("datasets/")
mlflow.log_artifact("train_utils/")
mlflow.log_artifact("transforms/")
mlflow.log_artifact("utils/")
mlflow.log_artifact("main.ipynb")

mlflow.end_run()